## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
data = pd.read_csv('/datasets/data.csv')
print(data.info())
print('')
print(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None

       children  days_employed  dob_years            education  education_id  \
0             1   -8437.673028         42               высшее             0   
1             1   -4024.803754         36              среднее             1   
2             0   -5623.422610         33              Среднее             1   
3 

### Вывод


1. Очевидные проблемы с регистром, немного NaN-ов
2. Самый странный столбец — трудовой стаж. Данные в нём представлены в трёх видах: NaN-ы, отрицательные числа и неадекатно большие числа. Отрицательные вполне могут быть ошибкой выгрузки, им я поменяю знак. С большими числами всё интереснее: я наблюдаю, что они встречаются, в основном, у людей старшего поколения, пенсионеров. При самом "трудоголичном" сценарии работы с 14 лет до 70, количество рабочих дней равно 20440, но никак не 340266. Однако, у меня есть гипотеза, что в этих  данных трудовой стаж почему-то представлен в часах, а не в днях. Для людей помладше, которые уже числятся пенсионерами, могла быть работа с льготным выходом на пенсию (год за два, например), либо пенсия по инвалидности. Итоговые значения в годах тогда выглядят правдоподобнее. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#print(data['children'].value_counts()) - проблема с количеством детей ( -1 и 20 детей). Заменю на 1 и 2 ребенка соответственно.
#print(data['dob_years'].value_counts()) - здесь есть строки с нулевым возрастом, их заменю средним возрастом
#print(data['gender'].value_counts()) - здесь есть непонятное значение, строку с ним удалим, т.к. среднее неприменимо
#print(data['family_status'].value_counts()) - нужно устранить разницу регистров
#print(data['income_type'].value_counts())
#print(data['purpose'].value_counts())
#print(data['total_income'].value_counts()) - пропущенные значения в зарплате заполню медианой
#print(data['days_employed'].value_counts()) - есть NaN, отрицательные числа, слишком большие числа
#print(data['education'].value_counts()) - тут есть дубликаты, но пропусков и странных данных нет


nan_total_income = data['total_income'].isna()
nan_days_employed = data['days_employed'].isna()

nan_check = np.all(nan_total_income.index == nan_days_employed.index)

#хочу заменить все NaN-ы нулями, но сначала проверю, нет ли в исходных данных нулей, чтобы их не изменить

check_null = (data.loc[data.loc[:, 'total_income'] == 0]['total_income'].count()) 

#print(check_null) нулей нет

data['total_income'] = data['total_income'].fillna(0)
check_date = data.loc[data.loc[:, 'total_income'] == 0]


# пропуски в одних и тех же строках, кроме этого никакой зависимости не неблюдаю,
#данные разнообразны в остальных столбцах. Заменяю NaN-ы нулями и в days_employed

data['days_employed'] = data['days_employed'].fillna(0)

# для дальнейших вычислений будет удобнее заменить нули медианами. Но вычислять её по отрицательным данным
# и неадекватно большим числам, думаю, некорректно. Сначала приведём имеющиеся данные в нормальный вид.
# заменяю большие числа на стаж в днях (а не в часах, по моей гипотезе), избавляюсь от минусов:

for element in data['days_employed']: 
    if element > 0:
        new_element = element / 24
        data['days_employed'] = data['days_employed'].replace(element, new_element)
for point in data['days_employed']:         
    if point < 0:
        new_point = point * -1
        data['days_employed'] = data['days_employed'].replace(point, new_point)
        
# теперь считаю медиану и заменяю в days_employed нули ею     
        
days_employed_median = data['days_employed'].median()        
data['days_employed'] = data['days_employed'].replace(0, days_employed_median)
#print (data['days_employed'])  

# аналогично поступаю с нулями в total_income

total_income_median = data['total_income'].median()        
data['total_income'] = data['total_income'].replace(0, total_income_median)
#print (data) 

#заменяю неадекватное количество детей

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
#print(data['children'].value_counts())

# заполняю нулевой возраст средним

dob_years_mean = data['dob_years'].mean()
data['dob_years'] = data['dob_years'].replace(0, dob_years_mean)
data['dob_years'] = data['dob_years'].astype('int')
#print(data['dob_years'].value_counts())

# удаляю строку со странным значением XNA в data['gender']

data = data.loc[data['gender'] != 'XNA']
#print(data['gender'].value_counts())

print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1  

### Вывод

NaN-ы были в столбцах с трудовым стажем и ежемесячным доходом. Я избавилась от пропусков — заполнила их медианой по этому столбцу. Медиану выбрала потому, что разброс значений по обоим столбцам существенный. NaN-ы были в одних и тех же строках - возможно, эти люди работают неофициально, либо не работают, либо это ошибка выгрузки. Пробелы в графах с возрастом заменила средним значением. Заменила странные значения в столбцах с количеством детей и полом на более правдоподобные.

### Замена типа данных

In [3]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
#print (data.info())
#print(data.head())

### Вывод

Просто перевела столбцы со стажем и зарплатой в целочисленный тип.

### Обработка дубликатов

In [4]:
data['family_status'] = data['family_status'].str.lower()
data['education'] = data['education'].str.lower()

#print(data['family_status'].value_counts())
#print(data['education'].value_counts())

data_dup = data.duplicated().sum()
print(data_dup)

data = data.drop_duplicates().reset_index(drop=True)

print('How many duplicates are there now?')
data_dup_check = data.duplicated().sum()
print(data_dup_check)

#print(data.info())


71
How many duplicates are there now?
0


### Вывод

Привела к единообразию разницы в регистрах с помощью метода .str.lower(). Они возникали, предположительно, из-за ручного ввода данных: кто-то вводит капсом, кто-то с большой буквы, кто-то прописными. Нужно либо проинструктировать всех сотрудников о способе представления данных, либо убрать ручной ввод и заменить его на выбор из выпадающего списка вариантов.

### Лемматизация

In [5]:
lemmas = {}
purposes = data['purpose'].unique()
for purpose in purposes:
    lemmas[purpose] = m.lemmatize(purpose)[::2]
#print(lemmas)    

lemmas_j = m.lemmatize(" ".join(data['purpose'].values))
lemmas_dict = Counter(lemmas_j)
print (lemmas_dict)


        

Counter({' ': 55021, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### Вывод

Создала словарь с леммами всех целей выдачи кредита, подсчитала количество лемм для дальнейшей категоризации

### Категоризация данных

In [6]:
# создаю перечень целей кредита с ключевыми словами:

realty_dict = ['недвижимость', 'жилье', 'жилой']
commerce_dict = ['коммерческий', 'сдача']
education_dict = ['образование']
auto_dict = ['автомобиль']
wedding_dict = ['свадьба']

def set_purpose_name(purpose):
    for purpose_name in lemmas[purpose]:
        if purpose_name in realty_dict:
            return 'realty'
        if purpose_name in commerce_dict:
            return 'commerce'
        if purpose_name in education_dict:
            return 'education'
        if purpose_name in auto_dict:
            return 'auto'
        if purpose_name in wedding_dict:
            return 'wedding'

data['purpose_name'] = data['purpose'].apply(set_purpose_name)
#print(data['purpose_name'].head(20))
#print(data['purpose'].head(20))
#print(data.head(15))

#ниже я определю класс дохода каждого человека исходя из его ежемесячного дохода - итого 6 классов

def income_class(income):
    if income < 100000:
        return '0 - 100 000'
    elif income < 200000:
        return '100 000 - 200 000'
    elif income < 300000:
        return '200 000 - 300 000'
    elif income < 400000:
        return '300 000 - 400 000'
    elif income < 500000:
        return '400 000 - 500 000'
    else:
        return 'more 500 000'
    
data['income_name'] = data['total_income'].apply(income_class)
#print(data['income_name'].head(20))
#print(data['total_income'].head(20))   
print(data['income_name'].value_counts())

100 000 - 200 000    11925
0 - 100 000           4463
200 000 - 300 000     3583
300 000 - 400 000      954
400 000 - 500 000      306
more 500 000           222
Name: income_name, dtype: int64


### Вывод

Я выделила пять категорий кредита: жилая недвижимость(она же - недвижимость без указанной цели), коммерческая недвижимость, автокредит, кредит на образование, кредит на свадьбу. Создала столбец purpose_name, где каждой цели кредита присвоила название категории.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data_pivot_children_sum = data.pivot_table(index='children', values = 'debt', aggfunc='sum')
data_pivot_children_count = data.pivot_table(index='children', values = 'debt', aggfunc='count')

data_pivot_children_sum['ratio'] =data_pivot_children_sum['debt'] / data_pivot_children_count['debt']
data_pivot_children = data_pivot_children_sum.drop(columns = 'debt')

def per(number):
    return '{:.2%}'.format(number)

data_pivot_children['ratio'] = data_pivot_children['ratio'].apply(per)

print(data_pivot_children)

#check_chill = (data.loc[data.loc[:, 'children'] == 5])
#print (check_chill)

          ratio
children       
0         7.54%
1         9.17%
2         9.49%
3         8.18%
4         9.76%
5         0.00%


### Вывод

Люди с пятью детьми не имеют задолженностей. Полагаю, что в сравнении с ответственностью за такое количество детей, ответственность за своевременную выплату - пустяк. Практически все они — люди семейные.
Бездетные люди, и имеющие троих детей выгоднее отличаются по количествам задолженностей от людей с 1, 2 или 4 детьми. Не думаю, что есть какая-то прямая взаимосвязь между количеством детей от 1 до 4, значения достаточно близки и нужно более глубокое изучение.
Бездетные выплачивают кредит лучше, чем те у кого от 1 до 4 детей. Вероятно, отсутствие трат на детей улучает их платежеспособность.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_pivot_family_sum = data.pivot_table(index='family_status', values = 'debt', aggfunc='sum')
data_pivot_family_count = data.pivot_table(index='family_status', values = 'debt', aggfunc='count')

data_pivot_family_sum['ratio'] =data_pivot_family_sum['debt'] / data_pivot_family_count['debt']
data_pivot_family = data_pivot_family_sum.drop(columns = 'debt')

data_pivot_family['ratio'] = data_pivot_family['ratio'].apply(per)
print(data_pivot_family)

                       ratio
family_status               
в разводе              7.11%
вдовец / вдова         6.57%
гражданский брак       9.35%
женат / замужем        7.55%
не женат / не замужем  9.75%


### Вывод

Больше всего задолженностей у незамужних людей и живущих в гражданском браке — вероятно, вступившие в официальный брак люди чувствуют на себе больше ответственности, чем незамужние. Свободные от брака люди проще относятся к обязательствам, они легче на подъём.

Женатые люди, как я сказала выше, более ответственны, кроме того, кредит может помогать выплачивать супруг / супруга. Наверняка и детей у этой категории людей больше, чем у незамужних. А дети тоже накладывают ответственность.

Меньше всего задолженностей у тех, кто потерял близкого человека: вдовы/вдовцы, люди в разводе. Полагаю, это связано с тем, что при потере партнёра они становятся рассудительнее и осторожнее- им теперь не на кого положиться. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_pivot_income_sum = data.pivot_table(index='income_name', values = 'debt', aggfunc='sum')
data_pivot_income_count = data.pivot_table(index='income_name', values = 'debt', aggfunc='count')

data_pivot_income_sum['ratio'] =data_pivot_income_sum['debt'] / data_pivot_income_count['debt']
data_pivot_income = data_pivot_income_sum.drop(columns = 'debt')

data_pivot_income['ratio'] = data_pivot_income['ratio'].apply(per)
print(data_pivot_income)

                   ratio
income_name             
0 - 100 000        7.93%
100 000 - 200 000  8.63%
200 000 - 300 000  7.03%
300 000 - 400 000  7.86%
400 000 - 500 000  5.56%
more 500 000       6.31%


### Вывод

Люди с высоким доходом (от 400 000 и более) имеют меньше всего задолженностей — это логично, их доход позволяет им выплачивать кредит вовремя. Хуже всего дело обстоит у людей с низким доходом (0 - 200 000), что тоже объяснимо. А вот в среднем классе (200 000 - 400 000) всё наоборот: люди, чей доход находится  у низшей границы среднего класса выплачивают кредит лучше, чем люди среднего класса с высоким доходом.

Можно заметить, что те, кто находятся у верхней границы своего класса дохода, имеет больше задолженностей, чем те, кто зарабатывает меньше их. Каждое второе значение в столбце ratio больше предыдущего. Казалось бы, у них больше возможностей для выплаты кредита и уровень задолженности должен падать, но нет. Думаю, здесь имеет влияние статусность. Люди стремятся перейти границу своего класса и выглядеть более успешными. Отсюда и больший процент задолженностей — машины, дома, свадьбы, которые люди себе не могут позволить, но хотят выглядеть так, будт могут.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#data_pivot_purpose = data.pivot_table(index=['purpose_name'], values='debt', aggfunc='sum')
#print(data_pivot_purpose)

data_pivot_purpose_sum = data.pivot_table(index='purpose_name', values = 'debt', aggfunc='sum')
data_pivot_purpose_count = data.pivot_table(index='purpose_name', values = 'debt', aggfunc='count')

data_pivot_purpose_sum['ratio'] =data_pivot_purpose_sum['debt'] / data_pivot_purpose_count['debt']
data_pivot_purpose = data_pivot_purpose_sum.drop(columns = 'debt')

data_pivot_purpose['ratio'] = data_pivot_purpose['ratio'].apply(per)
print(data_pivot_purpose)

              ratio
purpose_name       
auto          9.36%
commerce      7.55%
education     9.22%
realty        7.19%
wedding       8.00%


### Вывод

Меньше всего задолженностей по выплатам за жильё. Я считаю, это связано с тем, что категория достаточно обширная: люди могут брать кредит на  покупку дома, строительство дома, съём или покупку квартиры, на ипотеку, на ремонт. На ремонт кредиты меньше чем на покупку и постройку дома - их выплатить проще. Что касается долгосрочных кредитов на постройку или покупку жилья, здесь может быть ипотека. А в ипотеке обычно так: если нет просрочек, появляются бонусы вплоть до снижения ставки. А при наличии просрочек ставка увеличивается. Вот люди и стараются платить вовремя. 

Почти так же хорошо дело обстоит с коммерческой недвижимостью- вероятно потому, что с неё люди получают доход, которым выплачивают кредит. 

Кредиты за свадьбу выплатить немного труднее, чем за коммерческую недвижимость, но они, как правило, меньше кредитов на недвижимость по сумме и долг погасть легче. А ещё молодожёнам часто дарят деньги.

Кредиты на образование выплачиваются чуть лучше, чем кредиты на автомобиль - насколько мне известно, кредиты на образование можно начинать платить после окончания образования, т.е. обучившийся человек может найти работу и его выплачивать.

Автомобиль - недешёвое удовольствие (бензин, обслуживание, права, штрафы), соответственно, и денег у человека меньше. Эти кредиты выплачиваются хуже всего.

### Шаг 4. Общий вывод

Идеальный портрет человека, который хочет взять кредит, выглядит следующим образом: многодетный одинокий (потерявший супруга или в разводе) человек с высоким уровнем дохода, который собирается взять кредит для жилья или для бизнеса. Могу предполагать, что раз у этого человека высокий доход и много детей (которых он в состоянии обеспечить), он занимается преуспевающим бизнесом, у него есть "предпринимательская жилка", и он собирается развивать свой бизнес. 

Худший вариант заёмщика можно описать так: человек с несколькими детьми (от 1 до 4х детей), живущий в гражданском браке или вообще не замужем / не женат, с доходом чуть ниже среднего, но не очень низким. Он берёт кредит на автомобиль или образование. Его доходы не позволяют в полной мере оплатить кредит, он хочет изменить свою жизнь (получить образование и сменить род деятельности), либо автомобиль ему сильно нужен - это его единственный автомобиль, он нужен ему для работы, либо починить своё авто, без которого он обойтись не может. Ему хватает на жизнь, но на долгосрочные выплаты его текущий заработок не рассчитан.
